# Model Preparation

This notebook is dedicated at exploring some models, number of parameters, complexities, etc.

In [1]:
import numpy as np
import os
import sys
import torch
import torch.nn as nn
from torch.nn import functional as F
import thop


In [2]:
from model_blocks import *
from models import *
from utils import *
from tools import *

Loading faiss with AVX2 support.
Loading faiss.


In [3]:
from apex import amp

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
mixedblock = MixtureEncoderBlock().to(device)

In [6]:
count_parameters(mixedblock), count_trainable_parameters(mixedblock)

(3467520, 3467520)

In [5]:
count_parameters(mixedblock), count_trainable_parameters(mixedblock)

(3311616, 3311616)

In [6]:
# invect = torch.randint(0,2**10, (1024, 2048))
# invect = torch.randn((240, 2048, 192)).to(device) # 240 as batch size is the MAX it supports there in my memory... 
invect = torch.randn((24, 2048, 192)).to(device) 

In [7]:
%time macs, flops = thop.profile(mixedblock, inputs=(invect,))

torch.Size([24, 2048, 192]) torch.Size([24, 2048, 192])
CPU times: user 9.26 s, sys: 1.83 s, total: 11.1 s
Wall time: 2.88 s


In [9]:
macs, flops

(268066357248.0, 3262080.0)

In [8]:
mixedblock(invect)

torch.Size([24, 2048, 192]) torch.Size([24, 2048, 192])


tensor([[[-0.2500,  0.7942,  1.3775,  ...,  0.7410,  0.9121, -1.7052],
         [-1.6831,  1.7151,  0.3799,  ..., -0.4643,  0.5997,  1.0325],
         [-0.3742, -0.6832, -0.2251,  ..., -1.5051,  0.8147,  1.9625],
         ...,
         [-0.6992,  1.7105,  1.6438,  ..., -0.4991,  0.7874,  0.6929],
         [-0.3200, -0.0582, -0.1298,  ..., -1.0670,  1.0196,  1.2806],
         [-1.2842, -0.3365, -0.0657,  ...,  0.1423,  0.7543,  0.7420]],

        [[ 0.0194,  0.5308,  0.6158,  ..., -0.4822,  0.1210,  1.3848],
         [-1.6309,  1.6484,  0.0402,  ..., -1.2842,  1.1593, -0.5352],
         [-0.9461,  0.2626, -0.5823,  ..., -1.4153,  0.0558,  0.7368],
         ...,
         [ 0.2063,  0.5702, -0.0141,  ..., -0.0353,  0.4002,  0.4371],
         [-0.0747,  1.1403,  0.6513,  ..., -2.3313,  0.1766, -0.2762],
         [-0.5891,  0.3640,  0.9321,  ...,  0.5028,  0.0263,  0.7168]],

        [[-0.2645,  0.5276, -0.6710,  ..., -0.9991,  0.2293, -0.3468],
         [-1.9165,  1.1026,  2.4629,  ..., -0

In [8]:
# macs, flops

In [9]:
codebook = np.load("codes/utf8_3-seg_59328-codepoints_128-dim_N-37-k4_coprimes-(11, 13, 19, 23)_cycles-(11, 7, 4, 3)_dense.npy")
cm = ConvModel(codebook).to(device)

In [10]:
count_parameters(cm), count_trainable_parameters(cm)

(18405120, 18405120)

In [11]:
%%time 
invect = torch.randint(0,2**10, (24, 2048)).to(device)
macs, flops = thop.profile(cm, inputs=(invect,))
macs, flops

Either .total_ops or .total_params is already defined in Conv1d(512, 1024, kernel_size=(1,), stride=(1,)). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,), groups=4). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,), groups=4). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,), groups=4). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Dropout(p=0.3, inplace=False). Be careful, it might change your code's behavior.
Either .total_ops or .total_params is already defined in Conv1d(1024, 1024, kernel_size=(1,), stride=(1,)). Be careful, it might change your code's behavior.
Ei

CPU times: user 13 s, sys: 2.55 s, total: 15.6 s
Wall time: 4.01 s


/home/leo/venv3/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(932840472576.0, 18355584.0)

In [12]:
cm(invect)

tensor([[[0.7545, 0.2560, 0.6120,  ..., 0.2645, 0.8027, 0.2437],
         [0.7111, 0.2120, 0.7493,  ..., 0.3908, 0.8328, 0.1985],
         [0.7772, 0.2255, 0.6069,  ..., 0.3497, 0.7584, 0.2854],
         ...,
         [0.7864, 0.1760, 0.6395,  ..., 0.5127, 0.7975, 0.2253],
         [0.7336, 0.2480, 0.6397,  ..., 0.3461, 0.7581, 0.2547],
         [0.7859, 0.2410, 0.7199,  ..., 0.3973, 0.8293, 0.2470]],

        [[0.8164, 0.2982, 0.6592,  ..., 0.3190, 0.7663, 0.3769],
         [0.6774, 0.1635, 0.6392,  ..., 0.3516, 0.7301, 0.2013],
         [0.8025, 0.3236, 0.7460,  ..., 0.3783, 0.7612, 0.2891],
         ...,
         [0.7662, 0.2987, 0.6849,  ..., 0.2967, 0.7237, 0.3765],
         [0.6401, 0.2170, 0.7405,  ..., 0.3119, 0.7082, 0.2922],
         [0.7427, 0.2377, 0.6422,  ..., 0.3940, 0.8479, 0.2232]],

        [[0.7151, 0.2180, 0.7134,  ..., 0.3790, 0.7118, 0.4311],
         [0.7624, 0.1716, 0.6380,  ..., 0.3080, 0.8448, 0.3906],
         [0.6934, 0.2724, 0.5801,  ..., 0.3083, 0.7321, 0.